In [39]:
# Imports
import numpy as np
from pathlib import Path
from subcipher.mh import crack
from subcipher.io import export_result
from subcipher.codec import encrypt, decrypt # For creating a sample cipher if needed
from subcipher.text_utils import clean_text
from subcipher.alphabet import ALPHABET
import random

print("Imports successful.")

Imports successful.


In [40]:
# Load the reference transition matrix
model_path = Path("data/model/reference_tm.npy")
if not model_path.exists():
    print(f"Model file not found: {model_path}")
    print("Please ensure you have built and saved the reference_tm.npy first (e.g., using 02_bigram_model.ipynb or build_bigram_model.py).")
    raise FileNotFoundError(f"Model file not found: {model_path}. Please generate it first.")
else:
    tm_ref = np.load(model_path)
    print(f"Successfully loaded reference transition matrix from '{model_path}'. Shape: {tm_ref.shape}")


Successfully loaded reference transition matrix from 'data\model\reference_tm.npy'. Shape: (27, 27)


In [41]:
# Prepare Ciphertext
ciphertext_file_path = Path("data/test/text_1000_sample_1_ciphertext.txt") # The one we used
if ciphertext_file_path.exists():
    with open(ciphertext_file_path, 'r', encoding='utf-8') as f:
        raw_ciphertext = f.read()
    print(f"Loaded ciphertext from file: {ciphertext_file_path}. Length: {len(raw_ciphertext)}")
else:
    print(f"Ciphertext file {ciphertext_file_path} not found. Using a demo ciphertext.")
    true_plaintext_demo = "TOTO_JE_DEMONSTRACNI_TEXT_KTERY_BUDE_ZASIFROVAN_A_POTOM_PROLOMEN_POMOCI_METROPOLIS_HASTINGS_ALGORITMU_SNAD_TO_VYJDE_DOBRE"
    key_list_demo = list(ALPHABET)
    random.shuffle(key_list_demo)
    true_key_demo = "".join(key_list_demo)
    raw_ciphertext = encrypt(true_plaintext_demo, true_key_demo)
    print(f"Using demo ciphertext. True key for demo: {true_key_demo}")
    print(f"Demo Plaintext: {true_plaintext_demo}")

# Clean the ciphertext (important if it might contain chars not in ALPHABET)
ciphertext_to_crack = clean_text(raw_ciphertext)
print(f"Ciphertext to crack (cleaned, length {len(ciphertext_to_crack)}): {ciphertext_to_crack[:100]}...")


Ciphertext file data\test\text_1000_sample_1_ciphertext.txt not found. Using a demo ciphertext.
Using demo ciphertext. True key for demo: SARUFMVOGTDQEBWKHIYJX_NZLPC
Demo Plaintext: TOTO_JE_DEMONSTRACNI_TEXT_KTERY_BUDE_ZASIFROVAN_A_POTOM_PROLOMEN_POMOCI_METROPOLIS_HASTINGS_ALGORITMU_SNAD_TO_VYJDE_DOBRE
Ciphertext to crack (cleaned, length 121): JWJWCTFCUFEWBYJISRBGCJFZJCDJFILCAXUFCPSYGMIW_SBCSCKWJWECKIWQWEFBCKWEWRGCEFJIWKWQGYCOSYJGBVYCSQVWIGJE...


In [42]:
# Perform the cryptanalysis using crack()
iters = 20000  
temp = 1.0
seed = None 

print(f"\nStarting cryptanalysis with {iters} iterations...")
found_key, decrypted_plaintext, best_ll = crack(
    ciphertext_to_crack,
    tm_ref,
    iters=iters,
    temp=temp,
    seed=seed

)

print(f"\nCryptanalysis finished.")
print(f"Best Log-Likelihood: {best_ll:.4f}")
print(f"Found Key:           {found_key}")
print(f"\nDecrypted Plaintext (first 500 characters):")
print(decrypted_plaintext[:500])

if 'true_plaintext_demo' in locals() and true_plaintext_demo:
    matches = sum(p == t for p, t in zip(decrypted_plaintext, true_plaintext_demo))
    accuracy = matches / len(true_plaintext_demo) if len(true_plaintext_demo) > 0 else 0
    print(f"\n--- Demo Ciphertext Check ---")
    print(f"True Key for Demo:    {true_key_demo}")
    print(f"Accuracy with true demo plaintext: {accuracy:.2%}")
    if found_key == true_key_demo:
        print("Successfully recovered the true demo key!")
    else:
        print("Did not recover the exact true demo key.")



Starting cryptanalysis with 20000 iterations...

Cryptanalysis finished.
Best Log-Likelihood: -6.5891
Found Key:           FKYVED_TWONUGIAMXPSJQHBLRZC

Decrypted Plaintext (first 500 characters):
TITI_HA_LAEIWCTNSYWM_TAZT_FTANX_OQLA_RSCMPNIGSW_S_BITIE_BNIUIEAW_BIEIYM_EATNIBIUMC_JSCTMWDC_SUDINMTEQ_CWSL_TI_GXHLA_LIONA

--- Demo Ciphertext Check ---
True Key for Demo:    SARUFMVOGTDQEBWKHIYJX_NZLPC
Accuracy with true demo plaintext: 23.14%
Did not recover the exact true demo key.


In [43]:
# Export the results using export_result()

if ciphertext_file_path.exists() and "text_1000_sample_1" in str(ciphertext_file_path):
    output_length = 1000
    output_sample_id = 1
else: # Fallback for demo
    output_length = len(ciphertext_to_crack)
    output_sample_id = 99 

output_dest_dir = Path("exports/notebook_attack_demo")

print(f"\nExporting results to directory: {output_dest_dir}")
print(f"Using length={output_length}, sample_id={output_sample_id}")

export_result(
    plaintext=decrypted_plaintext,
    key=found_key,
    length=output_length,
    sample_id=output_sample_id,
    dest=output_dest_dir
)

print("\nResults exported successfully. Check the following files:")
print(f"Plaintext: {output_dest_dir / f'text_{output_length}_sample_{output_sample_id}_plaintext.txt'}")
print(f"Key:       {output_dest_dir / f'text_{output_length}_sample_{output_sample_id}_key.txt'}")

with open(output_dest_dir / f'text_{output_length}_sample_{output_sample_id}_plaintext.txt', 'r', encoding='utf-8') as f:
    print(f"\nContent of exported plaintext file (first 200 chars):\n{f.read(200)}...")



Exporting results to directory: exports\notebook_attack_demo
Using length=121, sample_id=99

Results exported successfully. Check the following files:
Plaintext: exports\notebook_attack_demo\text_121_sample_99_plaintext.txt
Key:       exports\notebook_attack_demo\text_121_sample_99_key.txt

Content of exported plaintext file (first 200 chars):
TITI_HA_LAEIWCTNSYWM_TAZT_FTANX_OQLA_RSCMPNIGSW_S_BITIE_BNIUIEAW_BIEIYM_EATNIBIUMC_JSCTMWDC_SUDINMTEQ_CWSL_TI_GXHLA_LIONA...
